In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [19]:
!pip install xgboost

In [33]:
import os
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal, spatial, stats
from fastdtw import fastdtw
from scipy.signal import find_peaks
import pywt
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from tensorflow.keras.layers import Input, Dense, Conv1D, Conv2D, MaxPooling2D, MaxPooling1D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.metrics import Accuracy, Precision, Recall
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import pickle
from xgboost import XGBClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [6]:
os.chdir('gdrive/My Drive/Weave/Earables Project/Final Experiments/')

In [7]:
data = pickle.load(open('data/all_features_df.pkl', 'rb'))
fold_idx = pickle.load(open('data/5_fold_idx.pkl', 'rb'))

In [8]:
labels = data.iloc[:,-3:-2]
ohe = OneHotEncoder()
ohe.fit(labels)

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=True)

In [42]:
subjects = list(np.unique(data.subject))
# print(subjects)
test_i = 7
test_sub = subjects[test_i]

sub_data = data[data.subject == test_sub]
test_session = '2'
test_data = sub_data[sub_data.session == test_session]

del subjects[test_i]
train_data = []
for sub in subjects:
    train_data.append(data[data.subject == sub])    
train_data = pd.concat(train_data)
train_sub_data = sub_data.drop(index = test_data.index)
train_data = pd.concat([train_data, train_sub_data])

X_train = train_data.iloc[:,:-3]
y_train = train_data.iloc[:,-3:-2]
X_test = test_data.iloc[:,:-3]
y_test = test_data.iloc[:,-3:-2]
y_train = ohe.transform(y_train).toarray()
y_test = ohe.transform(y_test).toarray()

In [37]:
imp_features = pickle.load(open('data/important_features.pkl', 'rb'))
X_train = X_train[imp_features]
X_test = X_test[imp_features]

In [43]:
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(92098, 852) (92098, 32)
(1550, 852) (1550, 32)


In [44]:
data.shape

(93648, 855)

In [45]:
n_classes = y_train.shape[1]
print('num. of classes: {}'.format(n_classes))

num. of classes: 32


In [ ]:
xgb_classifier = OneVsRestClassifier(XGBClassifier(verbose_eval=True, verbose=True, n_jobs=4))
xgb_classifier.fit(X_train, y_train)

In [ ]:
y_pred_imp_xg = xgb_classifier.predict(X_test)